In [3]:
import pandas as pd

dataset = pd.read_csv("opsd_raw.csv")

In [5]:

dataset.head()

,utc_timestamp,DK_load_actual_entsoe_transparency,DK_load_forecast_entsoe_transparency,DK_solar_capacity,DK_solar_generation_actual,DK_wind_capacity,DK_wind_generation_actual,DK_wind_offshore_capacity,DK_wind_offshore_generation_actual,DK_wind_onshore_capacity,...,GB_UKM_wind_capacity,GB_UKM_wind_generation_actual,GB_UKM_wind_offshore_capacity,GB_UKM_wind_offshore_generation_actual,GB_UKM_wind_onshore_capacity,GB_UKM_wind_onshore_generation_actual,GR_load_actual_entsoe_transparency,GR_load_forecast_entsoe_transparency,GR_solar_generation_actual,GR_wind_onshore_generation_actual
0,2015-01-01 00:00:00+00:00,NaN,NaN,489.0,NaN,4643.0,NaN,1264.0,NaN,3379.0,...,12214.0,NaN,4040.0,NaN,8174.0,385.0,NaN,NaN,NaN,NaN
1,2015-01-01 01:00:00+00:00,3100.02,3126.8,489.0,NaN,4643.0,2357.33,1264.0,902.71,3379.0,...,12214.0,1148.0,4040.0,117.0,8174.0,1041.0,5226.83,5550.0,NaN,555.0
2,2015-01-01 02:00:00+00:00,2980.39,3019.0,489.0,NaN,4643.0,2387.35,1264.0,830.87,3379.0,...,12214.0,1139.0,4040.0,120.0,8174.0,1020.0,4987.34,5251.0,NaN,557.0
3,2015-01-01 03:00:00+00:00,2933.49,2976.3,489.0,NaN,4643.0,2594.47,1264.0,915.43,3379.0,...,12214.0,1053.0,4040.0,115.0,8174.0,938.0,4879.48,5050.0,NaN,559.0
4,2015-01-01 04:00:00+00:00,2941.54,2982.5,489.0,0.01,4643.0,3092.01,1264.0,1056.28,3379.0,...,12214.0,1048.0,4040.0,114.0,8174.0,934.0,4909.53,5100.0,NaN,597.0
